In [ ]:
#구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

### 라이브러리

In [ ]:
import pandas as pd
import folium
import math

from itertools import combinations
from tqdm import tqdm
from typing import List

### 데이터

In [ ]:
bus = pd.read_csv('/content/drive/MyDrive/응통 분석 공모전/P-Median_데이터/bus_dong.csv')
bus = bus[['region','좌표Y','좌표X']]
bus.columns = ['name','latitude','longitude']

sub = pd.read_csv('/content/drive/MyDrive/응통 분석 공모전/P-Median_데이터/sub_dong.csv')
sub = sub[['region','경도','위도']]
sub.columns = ['name','latitude','longitude']

cafe = pd.read_csv('/content/drive/MyDrive/응통 분석 공모전/P-Median_데이터/cafe_dong.csv')
cafe = cafe[['동','위도','경도']]
cafe.columns = ['name','latitude','longitude']

conv = pd.read_csv('/content/drive/MyDrive/응통 분석 공모전/P-Median_데이터/conv_dong.csv')
conv = conv[['동','위도','경도']]
conv.columns = ['name','latitude','longitude']

In [ ]:
#여의동을 예시로 하는 것: 타 지역은 아래 주석 처리된 데이터 이용
region = pd.read_excel('/content/drive/MyDrive/응통 분석 공모전/P-Median_데이터/여의동.csv',encoding='cp949')
#가산동.xlsx / 서초3동.csv / 여의동.csv / 서교동.csv / 종로구.csv

In [ ]:
def varToRegion(df, region):
  dfNew = df[df['name']==region]
  return dfNew

#역삼1동 / 2:가산동 / 3:서초3동 / 4:여의동 / 5:서교동 / 6:종로1.2.3.4가동
busDong = varToRegion(bus, '여의동')
subDong = varToRegion(sub, '여의동')
convDong = varToRegion(conv, '여의동')
cafeDong = varToRegion(cafe, '여의동')
#분석대상 지역을 '여의동' 대신 입력

In [ ]:
region = region[['loc1','위도',	'경도']]
region.columns = ['name','latitude','longitude']

### 함수

In [ ]:
import math
from math import sin, cos, acos

#두 좌표 사이 거리 구하기

PI = 3.1415926535
def rToD(rad):
  return (rad*180/PI)

def getD(lat1, lon1, lat2,lon2):
  theta = lon1 - lon2
  dist = sin(rToD(lat1))*sin(rToD(lat2))+cos(rToD(lat1))*cos(rToD(lat2))*cos(rToD(theta))
  dist = acos(dist)
  dist = rToD(dist)
  dist = dist*60*1.1515
  dist = dist * 1.609344 * 1000 #meter
  return dist

In [ ]:
def p_list_set(distance_data: pd.DataFrame, p: int) -> List[List]:
    col_sum = list(distance_data.sum(axis=0))

    col_sum_tuple = [] 
    for i in range(len(col_sum)):
        tup = (i, col_sum[i])
        col_sum_tuple.append(tup)

    col_sum_tuple.sort(key=lambda x: x[1])
    col_sum_tuple = col_sum_tuple[: 2 * p] 

    p_list_set = [col_sum_tuple[i : i + p] for i in range(p)]

    return p_list_set

In [ ]:
def candidate_place(
    pb: pd.DataFrame, distance: pd.DataFrame, p_list_set: List[List]
) -> List:
    min_sum_list = []  
    for i in range(len(p_list_set)):
        tup_check = []
        for j in p_list_set[i]:
            tup_check.append(f"F_{j[0]}")
        check_df = distance[tup_check]

        check_df["min"] = 0  
        for k in range(len(check_df)):
            k_th_row = check_df.iloc[k][:-1] 
            check_df["min"][k] = min(k_th_row)

        min_sum_value = sum(check_df["min"])
        min_sum_list.append(min_sum_value)

    final_index = min_sum_list.index(min(min_sum_list))
    final_set = p_list_set[final_index]
    final_set.sort(key=lambda x: x[0])

    final_idx = [idx for idx, dist in final_set]
    final_market_data = pb.iloc[final_idx, :]
    final_market_data.reset_index(drop=True, inplace=True)

    name_list = [name for name in final_market_data["name"]]
    return name_list

In [ ]:
def top_value(char_list: List) -> int:
    appearance_candidate = list(pd.Series(char_list).value_counts())

    num_of_candidate = 38 #각 지역의 쓰레기통 개수 입력
    for first, second in zip(appearance_candidate[2:], appearance_candidate[3:]):
        if first == second:
            num_of_candidate += 1
        else:
            break

    return num_of_candidate

In [ ]:
def make_finalset(market_data: pd.DataFrame, char_list: List) -> pd.DataFrame:
    num_of_candidate = top_value(char_list)
    final_name_list = pd.Series(char_list).value_counts().index[:num_of_candidate]

    market_index = [
        (market_data[market_data["name"] == name].index)[0] for name in final_name_list
    ]

    market_final = market_data.iloc[market_index, :]
    market_final.reset_index(drop=True, inplace=True)
    return market_final

In [ ]:
def makeDf(df1, df2):
  F_list = []
  L_list = []
  for i in range(len(df1)):
    name = f"F_{i}"
    F_list.append(name)

  for i in range(len(df2)):
    name = f"L_{i}"
    L_list.append(name)
  df = pd.DataFrame(columns=F_list, index=L_list)
  return df

### p-median

In [ ]:
busDongregion = makeDf(region, busDong)
subDongregion = makeDf(region, subDong)
cafeDongregion = makeDf(region, cafeDong)
convDongregion = makeDf(region, convDong)

In [ ]:
def getDist(df2, df1, df):
  for i in range(len(df1)):
    for j in range(len(df2)):
      dist = getD(df1.iloc[i,1], df1.iloc[i,2], df2.iloc[j,1], df2.iloc[j,2])
      df.iloc[i,j] = dist
  return df

In [ ]:
busDongregion = getDist(region, busDong,busDongregion)
subDongregion = getDist(region, subDong,subDongregion)
cafeDongregion = getDist(region, cafeDong,cafeDongregion)
convDongregion = getDist(region, convDong,convDongregion)

In [ ]:
def getCandidate(df1, df2):
  char_list = []
  for p in range(3, 38) : #(3,각 지역의 쓰레기통 개수) 입력
    p_list = p_list_set(df2, p)
    name_list = candidate_place(df1, df2, p_list)

    char_list.extend(name_list)
  return char_list

In [ ]:
busDongList = getCandidate(region, busDongregion)
subDongList = getCandidate(region, subDongregion)
cafeDongList = getCandidate(region, cafeDongregion)
convDongList = getCandidate(region, convDongregion)

In [ ]:
busDongfinal = make_finalset(region, busDongList)
subDongfinal = make_finalset(region, subDongList)
cafeDongfinal = make_finalset(region, cafeDongList)
convDongfinal = make_finalset(region, convDongList)

In [ ]:
def toFinal(df1, df2, df):
  for i in range(len(df)):
    k = df['name'][i]
    for j in range(len(df1)):
      if df1['name'][j]==k:
        df['latitude'][i] = df1['latitude'][j]
        df['longitude'][i] = df1['longitude'][j]
  return df

In [ ]:
busDongfinal = toFinal(region, busDongList, busDongfinal)
subDongfinal = toFinal(region, subDongList, subDongfinal)
cafeDongfinal = toFinal(region, cafeDongList, cafeDongfinal)
convDongfinal = toFinal(region, convDongList, convDongfinal)

In [ ]:
df = pd.merge(subDongfinal, busDongfinal)
df = pd.merge(cafeDongfinal, df)
df = pd.merge(convDongfinal, df)
df['score']=0

for i in range(len(df)):
  for j in range(len(busDongfinal)):
    if df['name'][i]==busDongfinal['name'][j]:
      df['score'][i]+=j+1
  for j in range(len(subDongfinal)):
    if df['name'][i]==subDongfinal['name'][j]:
      df['score'][i]+=j+1
  for j in range(len(cafeDongfinal)):
    if df['name'][i]==cafeDongfinal['name'][j]:
      df['score'][i]+=j+1
  for j in range(len(convDongfinal)):
    if df['name'][i]==convDongfinal['name'][j]:
      df['score'][i]+=j+1

df = df.sort_values(by='score')
df.to_csv('df.csv',encoding='utf-8-sig')
df